## Case Study: Prediction of Graduate Admissions

<img src="admission.jpg" width=500 height = 280/>

**Data to be used**: `Admission_Predict.csv`

Following is the description of columns in `Admission_Predict.csv` file:

<TABLE CAPTION="Mobile Dataset">
<TR><TD><B>Variable</B></TD><TD><B>Description</B></TD></TR>
<TR><TD>GRE Score</TD><TD>GRE Scores (out of 340)</TD></TR>
<TR><TD>TOEFL Score</TD><TD>TOEFL Scores (out of 120)</TD></TR>
<TR><TD>University Rating</TD><TD>University Rating (out of 5)</TD></TR>
<TR><TD>SOP</TD><TD>Quality of Statement of Purpose (out of 5)</TD></TR>
<TR><TD>LOR</TD><TD>Quality of Letters of Recommendation (out of 5)</TD></TR>
<TR><TD>CGPA</TD><TD>Undergraduate GPA (out of 10)</TD></TR>   
<TR><TD>Research</TD><TD>Research Experience (either Yes or No)</TD></TR>
<TR><TD>Chances of Admit</TD><TD>Chance of Admit (ranging from 0 to 1)</TD></TR>
</TABLE>

### Import Libraries

In [2]:
import pandas as pd                  # Pandas
import numpy as np                   # Numpy
from matplotlib import pyplot as plt # Matplotlib

# Package to implement ML Algorithms
import sklearn
from sklearn.ensemble import RandomForestRegressor # Random Forest

# Import MAPIE to calculate prediction intervals
from mapie.regression import MapieRegressor

# To calculate coverage score
from mapie.metrics import regression_coverage_score

# Package for data partitioning
from sklearn.model_selection import train_test_split

# Package to record time
import time

# Module to save and load Python objects to and from files
import pickle 

# Ignore Deprecation Warnings
import warnings
warnings.filterwarnings('ignore')

# Display inline plots as vector-based (svg)
%config InlineBackend.figure_formats = ['svg']

%matplotlib inline

**Install MAPIE via `pip`**:

`!pip install mapie`

### Load Dataset

In [3]:
# Import Data
df = pd.read_csv('Admission_Predict.csv')
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,Yes,0.92
1,324,107,4,4.0,4.5,8.87,Yes,0.76
2,316,104,3,3.0,3.5,8.00,Yes,0.72
3,322,110,3,3.5,2.5,8.67,Yes,0.80
4,314,103,2,2.0,3.0,8.21,No,0.65


In [4]:
# Select input and output features
X = df.drop(columns = ['Chance of Admit'])
y = df['Chance of Admit']

In [5]:
# One hot encoding for categorical variables
X_encoded = pd.get_dummies(X)
X_encoded.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research_No,Research_Yes
0,337,118,4,4.5,4.5,9.65,False,True
1,324,107,4,4.0,4.5,8.87,False,True
2,316,104,3,3.0,3.5,8.00,False,True
3,322,110,3,3.5,2.5,8.67,False,True
4,314,103,2,2.0,3.0,8.21,True,False


In [6]:
# Data partitioning into train and test sets
train_X, test_X, train_y, test_y = train_test_split(X_encoded, y, test_size = 0.2, random_state = 42)

### Implement Random Forest Regressor

In [7]:
# Define your model
reg = RandomForestRegressor(random_state = 42)

In [8]:
# Fit the model
start = time.time()            # Start Time
reg.fit(train_X, train_y)  
stop = time.time()             # End Time
print(f"Training time: {stop - start}s")

Training time: 0.08753013610839844s


#### Evaluate Prediction Performance on Test Set

In [9]:
# Predict Test Set
y_pred = reg.predict(test_X)

# Evaluate the model on test set
r2 = sklearn.metrics.r2_score(test_y, y_pred)
print('R-squared on Test Set: %0.2f' %r2)

RMSE_test = sklearn.metrics.root_mean_squared_error(test_y, y_pred)
print('RMSE on Test Set: %0.2f' %RMSE_test)

R-squared on Test Set: 0.79
RMSE on Test Set: 0.07


#### Histogram of Residuals/Errors 

In [10]:
# Calculate the residuals by subtracting the predicted values from the actual test values
all_residuals = test_y - y_pred

# Set up the figure with custom size and resolution (DPI)
plt.figure(figsize=(6, 4), dpi = 150)

# Plot the histogram of residuals
plt.hist(all_residuals, bins = 25, color = 'lime', edgecolor = 'black')

# Label X and Y axes
plt.xlabel('Residuals', fontsize = 14)
plt.ylabel('# of Test Datapoints', fontsize = 14)

# Set the title of the plot
plt.title('Distribution of Residuals', fontsize = 16)

# Adjust the font size of x and y ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10);


plt.savefig('Residuals_Histogram.png', dpi=300)

#### Scatter Plot of Predicted Vs. Actual Values

In [11]:
# Setting the figure size and resolution
plt.figure(figsize = (6, 4), dpi = 150)

# Scatter plot of actual vs predicted values
plt.scatter(test_y, y_pred, color = 'blue', alpha = 0.6, edgecolor = 'black', s = 40)

# 45-degree reference line (perfect predictions)
plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], color = 'red', linestyle = '--', lw = 2)

# Axis labels and title
plt.xlabel('Actual Values', fontsize = 10)
plt.ylabel('Predicted Values', fontsize = 10)
plt.title('Predicted vs Actual Values', fontsize = 12)

# Adjust ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)


plt.savefig('Predicted_vs_Actual.png', dpi=300)

#### Feature Importance Plot

In [12]:
# Storing importance values from the trained model
importance = reg.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize=(8, 4))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['red', 'lime'])

plt.xlabel("Importance", fontsize = 12)
plt.ylabel("Input Feature", fontsize = 12)
plt.title('Which features are the most important for predicting admission chances?', fontsize = 12) 
plt.yticks(fontsize = 10) # fontsize of yticks
plt.xticks(fontsize = 10) # fontsize of xticks

plt.tight_layout();


plt.savefig('Feature_Importance.png', dpi=300)

### Prediction Intervals for Regression
<img src="Prediction_Interval.png" width="500" style="float: center"/>

#### **What is a Prediction Interval?**
- It is a **range of values** within which a new observation is expected to fall with a **certain probability**, given the existing data and model.

- **Probability**: The width of the prediction interval depends on the **desired confidence level**, (e.g., 95%), with higher confidence levels leading to wider intervals.

#### **Confidence Level of Prediction Interval**

- The confidence level of a prediction interval indicates the probability that the interval will contain the true value of the parameter being estimated.

- Mathematically, the confidence level of a prediction interval is denoted by $ (1 - \alpha) \times 100\% $, where $ \alpha $ is the significance level.

#### **Why Prediction Intervals are Useful?**

- **Uncertainty Quantification**: They provide a measure of the uncertainty in individual predictions, which is crucial for risk assessment and decision-making.

- **Communication**: They are an effective tool for communicating the uncertainty in predictions to stakeholders, making the model's predictions more interpretable.

#### **Prediction Intervals using MAPIE Regressor**

***MAPIE: Model Agnostic Prediction Interval Estimator***

- It is a Python library designed to estimate prediction intervals in a **model-agnostic way**.
- It can be used with **any machine learning model**, including linear models, decision trees, ensemble methods, and neural networks.

In [13]:
import os

os.cpu_count()

8

In [14]:
# Define MAPIE regressor
mapie = MapieRegressor(estimator = reg, # Prediction model to use
                       n_jobs = -1,
                       random_state = 42)

# Fit mapie regressor on training data
start = time.time()  
mapie.fit(train_X, train_y)
stop = time.time()             
print(f"Training time: {stop - start}s")

alpha = 0.1 # For 90% confidence level

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_X, alpha = alpha)

Training time: 2.614043951034546s


In [15]:
# Predicted values point prediction for all the students
y_test_pred

array([0.9375, 0.7902, 0.6137, 0.7004, 0.7919, 0.8781, 0.4647, 0.6347,
       0.8467, 0.7932, 0.7067, 0.723 , 0.6763, 0.9185, 0.8602, 0.564 ,
       0.8328, 0.5878, 0.5516, 0.561 , 0.6436, 0.5308, 0.7082, 0.7635,
       0.7712, 0.6155, 0.9364, 0.8334, 0.6637, 0.7318, 0.562 , 0.7264,
       0.5387, 0.8775, 0.6453, 0.7469, 0.5991, 0.9412, 0.6392, 0.6903,
       0.9551, 0.5558, 0.7019, 0.8299, 0.9385, 0.6326, 0.9423, 0.807 ,
       0.7592, 0.9295, 0.8965, 0.5392, 0.672 , 0.4751, 0.9423, 0.6361,
       0.962 , 0.7344, 0.6335, 0.4341, 0.6087, 0.7048, 0.6165, 0.5122,
       0.4714, 0.5794, 0.8997, 0.9066, 0.6973, 0.6971, 0.6098, 0.748 ,
       0.6987, 0.5829, 0.5691, 0.6017, 0.8311, 0.8662, 0.5127, 0.6647,
       0.6627, 0.878 , 0.6393, 0.8402, 0.7232, 0.6642, 0.6307, 0.7493,
       0.7225, 0.6977, 0.7595, 0.9241, 0.9135, 0.6989, 0.7702, 0.4614,
       0.6965, 0.7457, 0.7211, 0.6829])

In [16]:
# Prediction Intervals for all the students
y_test_pis

array([[[0.8246],
        [1.0491]],

       [[0.6705],
        [0.8945]],

       [[0.4883],
        [0.7132]],

       [[0.5926],
        [0.8242]],

       [[0.6818],
        [0.911 ]],

       [[0.7648],
        [0.9885]],

       [[0.354 ],
        [0.5928]],

       [[0.5204],
        [0.7521]],

       [[0.7312],
        [0.9602]],

       [[0.6715],
        [0.9002]],

       [[0.6035],
        [0.8309]],

       [[0.6086],
        [0.8344]],

       [[0.5552],
        [0.7863]],

       [[0.8109],
        [1.036 ]],

       [[0.7451],
        [0.9705]],

       [[0.4201],
        [0.655 ]],

       [[0.7151],
        [0.9382]],

       [[0.485 ],
        [0.7107]],

       [[0.4269],
        [0.6556]],

       [[0.4295],
        [0.6582]],

       [[0.5351],
        [0.7644]],

       [[0.4021],
        [0.6399]],

       [[0.5996],
        [0.8255]],

       [[0.6474],
        [0.8739]],

       [[0.6486],
        [0.8755]],

       [[0.5017],
        [0.7305]],

       [[0.8

In [17]:
# Storing results in a dataframe
predictions = test_y.to_frame()
predictions.columns = ['Actual Value']
predictions["Predicted Value"] = y_test_pred.round(2)
predictions["Lower Value"] = y_test_pis[:, 0].round(2)
predictions["Upper Value"] = y_test_pis[:, 1].round(2)

# Take a quick look
predictions.tail(5)

,Actual Value,Predicted Value,Lower Value,Upper Value
347,0.42,0.46,0.35,0.58
86,0.72,0.70,0.59,0.82
75,0.72,0.75,0.62,0.85
438,0.67,0.72,0.61,0.83
15,0.54,0.68,0.57,0.79


### **Coverage Calculation**
- **Coverage** refers to the proportion of true/actual values that fall within the prediction intervals generated by a model.

- It is a measure of how well the prediction intervals capture the actual values.

  $\text{Coverage} = \frac{\text{Number of actual values within prediction intervals}}{\text{Total number of actual values}}$

In [18]:
coverage = regression_coverage_score(test_y,           # Actual values
                                     y_test_pis[:, 0], # Lower bound of prediction intervals
                                     y_test_pis[:, 1]) # Upper bound of prediction intervals

coverage_percentage = coverage * 100
print(f"Coverage: {coverage_percentage:.2f}%")

Coverage: 92.00%


In [19]:
# Sort the predictions by 'Actual Value' for better visualization and reset the index
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

# Create a figure and axis object with specified size and resolution
fig, ax = plt.subplots(figsize=(8, 4))

# Plot the actual values with green dots
plt.plot(sorted_predictions["Actual Value"], 'go', markersize=3, label="Actual Value")

# Fill the area between the lower and upper bounds of the prediction intervals with semi-transparent green color
plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.2, color="green", label="Prediction Interval")

# Set font size for x and y ticks
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Set the limit for the x-axis to cover the range of samples
plt.xlim([0, len(sorted_predictions)])

# Label the x-axis and y-axis with appropriate font size
plt.xlabel("Samples", fontsize=10)
plt.ylabel("Target", fontsize=10)

# Add a title to the plot, including the coverage percentage, with bold formatting
plt.title(f"Prediction Intervals and Coverage: {coverage_percentage:.2f}%", fontsize=12, fontweight="bold")

# Add a legend to the plot, placed in the upper left, with specified font size
plt.legend(loc="upper left", fontsize=10);


plt.savefig('Prediction_Intervals.png')

### Save Model

In [23]:
# Creating the file where we want to write the model
reg_pickle = open('reg_admission.pickle', 'wb') 

# Write RF model to the file
pickle.dump(mapie, reg_pickle)

# Close the file
reg_pickle.close()

Exception ignored in: <function ResourceTracker.__del__ at 0x1107cdc60>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x105751c60>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x10503dc60>
Traceback (most recent call last